In [15]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=16, app_name='west0_mclaren_2', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 16 executors with 48G RAM each


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34353)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_mclaren_2...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [20]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [81]:
seasonal_token = load_schema.lobby(spark, "pc", "live", "CurrencyIncreased", "2022-09-07", "2022-11-08").where(col("Currency").like("%seasonaltoken%"))

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34639)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:34639)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [82]:
seasonal_token = seasonal_token.where(~col("Reason").isin(["pas 기능 테스트_220929", "pas 테스트"])).where(col("Amount") > 0)

In [83]:
seasonal_token_by_user = seasonal_token.groupBy("AccountId").agg(sum("Amount").alias("token_amount")).withColumnRenamed("AccountId", "account_id")

In [88]:
mclaren_seg_pre_and_post_info = mclaren_seg_pre_and_post_info.join(seasonal_token_by_user, "account_id", "left")

In [95]:
mysql.insert_table(mclaren_seg_pre_and_post_info, "labs", "mclaren_seg_pre_and_post_info")

itemdesc.14300003	McLAREN LOOT CACHE  
itemdesc.14300004	McLAREN PRIME PARCEL  
itemdesc.14300005	McLAREN EVENT LOOT CACHE

In [16]:
crate_increased = load_schema.lobby(spark, "pc", "live", "WorkshopCrateIncreased", "2022-09-07", "2022-11-08")

In [17]:
prime_parcel_increased = crate_increased.where(col("ItemDescId") == "itemdesc.14300004")

In [18]:
prime_parcel_increased = prime_parcel_increased.withColumnRenamed("AccountId", "account_id")

In [19]:
prime_parcel_increased.where(col("Amount") > 0).groupBy("Reason").agg(count("*")).show(truncate=False)

+----------------+--------+
|Reason          |count(1)|
+----------------+--------+
|buy-cash-item   |238961  |
|by-support      |5       |
|open-crate-bonus|405327  |
+----------------+--------+



In [6]:
user_group = mysql.read_table(spark, "labs", "mclaren_seg_pre_and_post_info").select("group", "mclaren_group", "account_id")

In [7]:
m1_sb_user = user_group.where((col("group") == "M-1") & (col("mclaren_group") == "SB"))

In [11]:
m1_sb_prime_parcel_increased = prime_parcel_increased.join(m1_sb_user, "account_id")

In [13]:
m1_sb_prime_parcel_increased.where(col("Amount") > 0).groupBy("Reason").agg(countDistinct("account_id"), count("*")).show(truncate=False)

+----------------+--------------------------+--------+
|Reason          |count(DISTINCT account_id)|count(1)|
+----------------+--------------------------+--------+
|by-support      |1                         |1       |
|open-crate-bonus|7997                      |9022    |
+----------------+--------------------------+--------+



In [60]:
crate_open = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", "2022-09-07", "2022-11-08")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37515)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:37515)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [61]:
season_crate_open = crate_open.where(col("ItemDescId").isin(["itemdesc.14300003", "itemdesc.14300004", "itemdesc.14300005"]))

In [62]:
season_crate_open = season_crate_open.withColumn("loot_cache_open", when(col("ItemDescId") == "itemdesc.14300003", col("OpenAmount")).otherwise(lit(None))) \
    .withColumn("prime_parcel_open", when(col("ItemDescId") == "itemdesc.14300004", col("OpenAmount")).otherwise(lit(None))) \
    .withColumn("event_loot_cache_open", when(col("ItemDescId") == "itemdesc.14300005", col("OpenAmount")).otherwise(lit(None)))

In [63]:
season_crate_open_by_user = season_crate_open.groupBy("AccountId").agg(sum("loot_cache_open").alias("loot_cache_open_cnt"), \
                                                                       sum("prime_parcel_open").alias("prime_parcel_open_cnt"), \
                                                                       sum("event_loot_cache_open").alias("event_loot_cache_open_cnt")
                                                                      ).withColumnRenamed("AccountId", "account_id")

In [74]:
mclaren_seg_pre_and_post_info = mysql.read_table(spark, "labs", "mclaren_seg_pre_and_post_info2")
mclaren_seg_pre_and_post_info = mclaren_seg_pre_and_post_info.drop("event_loot_cache_open_cnt")
season_crate_open_by_user = season_crate_open_by_user.select("account_id", "event_loot_cache_open_cnt")

In [75]:
mclaren_seg_pre_and_post_info = mclaren_seg_pre_and_post_info.join(season_crate_open_by_user, "account_id", "left")

In [76]:
# mysql.drop_table("labs", "mclaren_seg_pre_and_post_info")
mysql.insert_table(mclaren_seg_pre_and_post_info, "labs", "mclaren_seg_pre_and_post_info")

In [31]:
# mclaren 상품에 gcoin 사용 후 mclaren을 획득하는데까지 걸리는 날짜수 (접속일 기준)

In [20]:
user_login = load_data_mart('pc', "2022-09-07", "2022-11-08", "user_master").where(col("lastlogindate") == col("date")).withColumnRenamed("accountid", "account_id")

In [21]:
mclaren_gcoin = load_data_mart('pc', "2022-09-07", "2022-11-02", "gcoin_use").where(col("event_name") == "202209_season_workshop") \
    .groupBy("account_id").agg(min("date").alias("mclaren_gcoin_first_date"))

In [22]:
user_login_after_mclaren_gcoin = user_login.join(mclaren_gcoin, "account_id", "left").where(col("date") >= col("mclaren_gcoin_first_date"))

In [23]:
from pyspark.sql.window import Window
Window_Spec  = Window.partitionBy("account_id").orderBy("date")
login_date_num_after_use_mclaren_gcoin = user_login_after_mclaren_gcoin.withColumn("login_date_num_after_use_mclaren_gcoin", row_number().over(Window_Spec))

In [26]:
login_date_num_after_use_mclaren_gcoin = login_date_num_after_use_mclaren_gcoin.select("date", "account_id", "login_date_num_after_use_mclaren_gcoin")

In [24]:
mclaren_user = mysql.read_table(spark, "labs", "mclaren_owner")
mclaren_distinct_user = mclaren_user.groupBy("AccountId").agg(min("own_date").alias("first_own_date"))

In [27]:
mclaren_distinct_user = mclaren_distinct_user.withColumnRenamed("AccountId", "account_id").withColumnRenamed("first_own_date", "date")

In [28]:
mclaren_own_after_use_mclaren_gcoin_date_num = mclaren_distinct_user.join(login_date_num_after_use_mclaren_gcoin, \
                           ["account_id", "date"], "left")

In [33]:
mclaren_seg_pre_and_post_info = mclaren_seg_pre_and_post_info.join(mclaren_own_after_use_mclaren_gcoin_date_num.select("account_id", "login_date_num_after_use_mclaren_gcoin"), "account_id", "left")

In [36]:
season_crate_first_open_by_user = season_crate_open.groupBy("AccountId").agg(min("date").alias("first_crate_open_date")).withColumnRenamed("AccountId", "account_id")

In [38]:
user_login_after_first_crate_open = user_login.join(season_crate_first_open_by_user, "account_id", "left").where(col("date") >= col("first_crate_open_date"))

In [39]:
Window_Spec  = Window.partitionBy("account_id").orderBy("date")
login_date_num_after_first_crate_open = user_login_after_first_crate_open.withColumn("login_date_num_after_fisrt_crate_open", row_number().over(Window_Spec))

In [41]:
login_date_num_after_first_crate_open = login_date_num_after_first_crate_open.select("date", "account_id", "login_date_num_after_fisrt_crate_open")

In [42]:
mclaren_own_after_first_crate_open_date_num = mclaren_distinct_user.join(login_date_num_after_first_crate_open, \
                           ["account_id", "date"], "left")

In [43]:
mclaren_own_after_first_crate_open_date_num.printSchema()

root
 |-- account_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- login_date_num_after_fisrt_crate_open: integer (nullable = true)



In [44]:
mclaren_seg_pre_and_post_info = mclaren_seg_pre_and_post_info.join(mclaren_own_after_first_crate_open_date_num.select("account_id", "login_date_num_after_fisrt_crate_open"), "account_id", "left")

In [45]:
mysql.drop_table("labs", "mclaren_seg_pre_and_post_info2")
mysql.insert_table(mclaren_seg_pre_and_post_info, "labs", "mclaren_seg_pre_and_post_info2")

In [56]:
mysql.drop_table("labs", "mclaren_seg_pre_and_post_info")
mysql.insert_table(mclaren_seg_pre_and_post_info, "labs", "mclaren_seg_pre_and_post_info")